settings+functions

In [1]:
import numpy as np
from joblib import Parallel, delayed
from multiprocessing import Pool, cpu_count, Manager
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
from Gpack import read_pickle

cont_tab_and_grids=read_pickle("cont_tab_and_grids_test")
actions=cont_tab_and_grids['cont_tab']
costs=cont_tab_and_grids['costs']
pred_states=cont_tab_and_grids['HIOM results']


ore_points = np.round(np.arange(5000, 6001, 50), 2) 
sio2_points = np.linspace(2500, 10000, 10) 
cao_points = np.linspace(1.15, 1.40, 10) 
lime_points = np.round(np.linspace(0, 400, 20), 2) 
v235_points = np.round(np.linspace(0, 400, 20), 2) 

def trilinear_interpolation(point, V):
    x, y, z = point
    x0, y0, z0 = map(nearest_index, (sio2_points, cao_points, ore_points), point)
    x1, y1, z1 = x0 + 1, y0 + 1, z0 + 1

    # Ensure the indices are within bounds
    x1 = min(x1, len(sio2_points) - 1)
    y1 = min(y1, len(cao_points) - 1)
    z1 = min(z1, len(ore_points) - 1)

    c000 = V[x0, y0, z0]
    c001 = V[x0, y0, z1]
    c010 = V[x0, y1, z0]
    c011 = V[x0, y1, z1]
    c100 = V[x1, y0, z0]
    c101 = V[x1, y0, z1]
    c110 = V[x1, y1, z0]
    c111 = V[x1, y1, z1]

    xd = (x - sio2_points[x0]) / (sio2_points[x1] - sio2_points[x0]) if x0 != x1 else 0
    yd = (y - cao_points[y0]) / (cao_points[y1] - cao_points[y0]) if y0 != y1 else 0
    zd = (z - ore_points[z0]) / (ore_points[z1] - ore_points[z0]) if z0 != z1 else 0

    c00 = c000*(1-xd) + c100*xd
    c01 = c001*(1-xd) + c101*xd
    c10 = c010*(1-xd) + c110*xd
    c11 = c011*(1-xd) + c111*xd

    c0 = c00*(1-yd) + c10*yd
    c1 = c01*(1-yd) + c11*yd

    c = c0*(1-zd) + c1*zd

    return c

def nearest_index(array, value):
    """Find the index of the nearest value in the array to the given value."""
    return min(range(len(array)), key=lambda i: abs(array[i] - value))




original sequential method

In [ ]:
def perform_dynamic_programming(data_dict):
    # Define the convergence threshold
    threshold = 1
    max_iterations = 1000

    V = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=float)
    optimal_actions_1 = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=int)

    for iteration in range(max_iterations):
        V_prev = np.copy(V)

        for i, sio2 in enumerate(sio2_points):
            for j, cao in enumerate(cao_points):
                for k, ore in enumerate(ore_points):
                    valid_keys = [(sio2, cao, ore, lime) for lime in v235_points if (sio2, cao, ore, lime) in data_dict]
                    if not valid_keys:
                        continue

                    update_value_function(V, V_prev, optimal_actions_1, i, j, k,
                                    ore, valid_keys, data_dict)

        if np.all(np.abs(V_prev - V) < threshold):
            break
    print (iteration)

    return V, optimal_actions_1

def update_value_function(V, V_prev, optimal_actions_1, i, j, k, ore, valid_keys,
    data_dict):
    next_states = np.array([data_dict[key] for key in valid_keys])
    next_sio2_values, next_cao_values = next_states[:, 0], next_states[:, 1]

    # Compute costs using broadcasting
    costa = np.where(next_cao_values < 1.25, 400000 * (1.25 - next_cao_values)**2,
                    np.where(next_cao_values == 1.25, 0, 400000 * (1.25 -
                        next_cao_values)**2))

    # Estimate the value at the next state using interpolation
    next_values = np.array([trilinear_interpolation([next_sio2, next_cao, ore],
        V_prev) for next_sio2, next_cao in zip(next_sio2_values, next_cao_values)])
    costs = costa + 0.95 * next_values

    # Find the minimum cost and the corresponding action
    min_cost_idx = np.argmin(costs)
    min_cost = costs[min_cost_idx]

    # Update the value function and optimal action
    V[i, j, k] = min_cost
    optimal_actions_1[i, j, k] = valid_keys[min_cost_idx][3]

costs, action = perform_dynamic_programming(pred_states)

multiprocessing

In [2]:
"""results are correct but spend longer time than sequential version"""
def update_value_function(i, j, k, V_prev, data_dict):
    sio2 = sio2_points[i]
    cao = cao_points[j]
    ore = ore_points[k]

    valid_keys = [(sio2, cao, ore, lime) for lime in v235_points if (sio2, cao, ore, lime) in data_dict]

    next_states = np.array([data_dict[key] for key in valid_keys])
    next_sio2_values, next_cao_values = next_states[:, 0], next_states[:, 1]

    # Compute costs using broadcasting
    costa = np.where(next_cao_values < 1.25, 400000 * (1.25 - next_cao_values)**2,
                     np.where(next_cao_values == 1.25, 0, 400000 * (1.25 - next_cao_values)**2))

    # Estimate the value at the next state using interpolation
    next_values = np.array([trilinear_interpolation([next_sio2, next_cao, ore], V_prev) 
                            for next_sio2, next_cao in zip(next_sio2_values, next_cao_values)])
    costs = costa + 0.95 * next_values

    # Find the minimum cost and the corresponding action
    min_cost_idx = np.argmin(costs)
    min_cost = costs[min_cost_idx]
    optimal_action = valid_keys[min_cost_idx][3]

    return (i, j, k, min_cost, optimal_action)

def perform_dynamic_programming(data_dict):
    # Define the convergence threshold
    threshold = 1
    max_iterations = 1000

    # Initialize value function and optimal actions
    V = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=float)
    optimal_actions_1 = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=int)
    #all_points = [(i, j, k) for i in range(len(sio2_points)) for j in range(len(cao_points)) for k in range(len(ore_points))]
    for iteration in range(max_iterations):
        V_prev = np.copy(V)
        print(f"before pool started {iteration} iteration")
        # Use multiprocessing Pool for parallel processing
        with Pool(cpu_count()) as pool:
            #process_partial = partial(update_value_function, V_prev=V_prev, data_dict=data_dict)
            results = pool.starmap(update_value_function, [(i,j,k,V_prev,data_dict) for i in range(len(sio2_points)) for j in range(len(cao_points)) for k in range(len(ore_points))])
            #results = [pool.apply(update_value_function, args=(i,j,k,V_prev,data_dict)) for i in range(len(sio2_points)) for j in range(len(cao_points)) for k in range(len(ore_points))]
        print(f"pool finished {iteration} iteration")

        # Update value function and optimal actions with the results
        for (i, j, k, min_cost, optimal_action) in results:
            if min_cost is not None:
                V[i, j, k] = min_cost
                optimal_actions_1[i, j, k] = optimal_action
        print(f"update dinished {iteration} iteration")
        # Check for convergence
        if np.all(np.abs(V_prev - V) < threshold):
            print(f"Converged in {iteration + 1} iterations")
            break

    return V, optimal_actions_1

costs_new, action_new = perform_dynamic_programming(pred_states)

thread

In [2]:

""""fastest version but still a littl bit slower than sequential version"""
from concurrent.futures import ThreadPoolExecutor, as_completed
from concurrent.futures import ProcessPoolExecutor, as_completed
import cupy as cp 
def update_value_function(i, j, k, V_prev, data_dict):
    sio2 = sio2_points[i]
    cao = cao_points[j]
    ore = ore_points[k]

    valid_keys = [(sio2, cao, ore, lime) for lime in v235_points if (sio2, cao, ore, lime) in data_dict]
    if not valid_keys:
        return (i, j, k, None, None)

    next_states = np.array([data_dict[key] for key in valid_keys])
    next_sio2_values, next_cao_values = next_states[:, 0], next_states[:, 1]

    # Compute costs using broadcasting
    costa = np.where(next_cao_values < 1.25, 400000 * (1.25 - next_cao_values)**2,
                     np.where(next_cao_values == 1.25, 0, 400000 * (1.25 - next_cao_values)**2))

    # Estimate the value at the next state using interpolation
    next_values = np.array([trilinear_interpolation([next_sio2, next_cao, ore], V_prev) 
                            for next_sio2, next_cao in zip(next_sio2_values, next_cao_values)])
    costs = costa + 0.95 * next_values

    # Find the minimum cost and the corresponding action
    min_cost_idx = np.argmin(costs)
    min_cost = costs[min_cost_idx]
    optimal_action = valid_keys[min_cost_idx][3]

    return (i, j, k, min_cost, optimal_action)

def perform_dynamic_programming(data_dict):
    # Define the convergence threshold
    threshold = 1
    max_iterations = 1000

    # Initialize value function and optimal actions
    V = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=float)
    optimal_actions_1 = np.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=int)

    for iteration in range(max_iterations):
        V_prev = np.copy(V)

        # Create a list of all points
        all_points = [(i, j, k) for i in range(len(sio2_points)) for j in range(len(cao_points)) for k in range(len(ore_points))]

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
            future_to_point = {executor.submit(update_value_function, i, j, k, V_prev, data_dict): (i, j, k) for i, j, k in all_points}
            for future in as_completed(future_to_point):
                i, j, k, min_cost, optimal_action = future.result()
                if min_cost is not None:
                    V[i, j, k] = min_cost
                    optimal_actions_1[i, j, k] = optimal_action

        # Check for convergence
        if np.all(np.abs(V_prev - V) < threshold):
            print(f"Converged in {iteration + 1} iterations")
            break

    return V, optimal_actions_1

costs_new, action_new = perform_dynamic_programming(pred_states)

Converged in 60 iterations


GPU based on the thread method

In [ ]:
def update_value_function(i, j, k, V_prev, data_dict):
    sio2 = sio2_points[i]
    cao = cao_points[j]
    ore = ore_points[k]

    valid_keys = [(sio2, cao, ore, lime) for lime in v235_points if (sio2, cao, ore, lime) in data_dict]
    if not valid_keys:
        return (i, j, k, None, None)

    next_states = cp.array([data_dict[key] for key in valid_keys])
    next_sio2_values, next_cao_values = next_states[:, 0], next_states[:, 1]

    # Compute costs using broadcasting
    costa = cp.where(next_cao_values < 1.25, 400000 * (1.25 - next_cao_values)**2,
                        cp.where(next_cao_values == 1.25, 0, 400000 * (1.25 - next_cao_values)**2))

    # Estimate the value at the next state using interpolation
    next_values = cp.array([trilinear_interpolation([next_sio2, next_cao, ore], V_prev) 
                            for next_sio2, next_cao in zip(next_sio2_values, next_cao_values)])
    costs = costa + 0.95 * next_values

    # Find the minimum cost and the corresponding action
    min_cost_idx = cp.argmin(costs)
    min_cost = costs[min_cost_idx]
    optimal_action = valid_keys[min_cost_idx][3]

    return (i, j, k, min_cost, optimal_action)

def perform_dynamic_programming(data_dict):
    # Define the convergence threshold
    threshold = 1
    max_iterations = 1000

    # Initialize value function and optimal actions
    V = cp.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=float)
    optimal_actions_1 = cp.zeros((len(sio2_points), len(cao_points), len(ore_points)), dtype=int)

    for iteration in range(max_iterations):
        V_prev = cp.copy(V)

        # Create a list of all points
        all_points = [(i, j, k) for i in range(len(sio2_points)) for j in range(len(cao_points)) for k in range(len(ore_points))]

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
            future_to_point = {executor.submit(update_value_function, i, j, k, V_prev, data_dict): (i, j, k) for i, j, k in all_points}
            for future in as_completed(future_to_point):
                i, j, k, min_cost, optimal_action = future.result()
                if min_cost is not None:
                    V[i, j, k] = min_cost
                    optimal_actions_1[i, j, k] = optimal_action

        
        # Check for convergence
        if cp.all(cp.abs(V_prev - V) < threshold):
            print(f"Converged in {iteration + 1} iterations")
            break

    return V, optimal_actions_1

